# RAG

Let's evaluate your architecture on a Q&A dataset for the LangChain python docs.

Common RAG architectures have two main components:
1. Retriever -> provides information from a knowledge base. Vector search is simple and powerful, but this can include any database or arbitrary search engine
2. Response generator -> synthesizes a response to the user input based on a mixture of learned knowledge and the retrieved input.

Focusing on retrievers for unstructured data: you still have some additional design decisions you may want to make:

- What chunk size(s) to use for each document: too large and your system will be able to consider fewer documents at a time. Too small and the chunks themselves lack important context needed to interpret their content.
- How to index a single chunk: generating a single vector from an embedding model may be fine, or you can generate additional vectors based on summaries, hypothetical questions, or other related content. Some may even consider incorporating a keyword index or other structured metadata to better support different types of searches.
- How to assemble the retrieved chunks: once you've fetched the k-best list of "relevant" documents, you may want to do things like:
  - re-integrate the document into its parent context.
  - rerank the documents based on other criteria
 
All of these options come with tradeoffs in cost, response quality, and time. This may seem overwhelming at first! The good news is that the retrieval and response mechanism can be modular -> the better the information, the better the response, and the better the LLM, the better it is able to integrate the knowledge.

This notebook provides a RAG gym/playground you can use to evaluate different RAG strategies on a Q&A dataset generated from LangChain's python docs. The intent is to make it easy to experiment with different techniques to see their tradeoffs and make the appropriate decision for your use case.

## Pre-requisites

We will install quite a few prerequisites for this example since we are comparing various techinques and models.

In [ ]:
# %pip install -U langchain_benchmarks
# %pip install -U langchain langsmith langchainhub chromadb openai huggingface pandas langchain_experimental

In [1]:
%load_ext autoreload
%autoreload 2

For this code to work, please configure LangSmith environment variables with your credentials.

In [2]:
import os

os.environ[
    "LANGCHAIN_ENDPOINT"
] = "http://localhost:1984"  # "https://api.smith.langchain.com
os.environ["LANGCHAIN_API_KEY"] = "sk-..."  # Your API key

# Silence warnings from HuggingFace
os.environ["TOKENIZERS_PARALLELISM"] = False

## Review Q&A "environments"

The registry provides configurations to test out common architectures on curated datasets.

In [3]:
from langchain_benchmarks import clone_public_dataset
from langchain_benchmarks.rag import registry

In [4]:
registry

ID,Name,Dataset ID,Description
0,LangChain Docs Q&A,452ccafc-18e1-4314-885b-edd735f17b9d,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).


In [5]:
langchain_docs = registry[0]
langchain_docs

ID,0
Name,LangChain Docs Q&A
Dataset ID,452ccafc-18e1-4314-885b-edd735f17b9d
Description,Questions and answers based on a snapshot of the LangChain python docs. The environment provides th...
Retriever Factories,"basic, parent-doc, hyde"
Architecture Factories,conversational-retrieval-qa


In [6]:
clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)

Dataset LangChain Docs Q&A already exists. Skipping.
You can access the dataset at http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b.


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

retriever_factory = langchain_docs.retriever_factories["basic"]
# Indexes the documents with the specified embeddings
# Note that this does not apply any chunking to the docs,
# which means the documents can be of arbitrary length
retriever = retriever_factory(embeddings)

In [9]:
# Factory for creating a conversational retrieval QA chain

chain_factory = langchain_docs.architecture_factories["conversational-retrieval-qa"]

In [10]:
from langchain.chat_models import ChatAnthropic

# Example
llm = ChatAnthropic(model="claude-2", temperature=1)

chain_factory(retriever, llm=llm).invoke({"question": "what's lcel?"})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


" LCEL (LangChain Expression Language) is a declarative way to easily compose chains together in Langchain. Here's a brief 80 word summary:\n\nLCEL lets you build chains for NLP tasks like Question Answering by composing together Runnables - reusable building blocks. It supports streaming, parallelism, retries, and more. Chains built with LCEL integrate seamlessly with LangSmith for observability and LangServe for production deployment. LCEL makes it easy to go from prototype to production with no code change. Key features include performance optimizations, access to intermediate results, and input/output validation via schemas. [0][1][2][3]"

### Evaluate

Let's evaluate a retriever now.

In [11]:
from functools import partial

from langchain_benchmarks.rag import RAG_EVALUATION
from langsmith.client import Client

In [12]:
client = Client()

In [13]:
test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, retriever, llm=llm),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-essential-wood-50' at:
http://localhost/o/00000000-0000-0000-0000-000000000000/projects/p/77aefb67-1c66-45e0-a508-cea0faaf30c1?eval=true

View all tests for Dataset LangChain Docs Q&A at:
http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b
[------------------------------------------------->] 86/86

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).



 Eval quantiles:
        embedding_cosine_distance  faithfulness  score_string:accuracy error  \
count                   86.000000     86.000000              86.000000     0   
unique                        NaN           NaN                    NaN     0   
top                           NaN           NaN                    NaN   NaN   
freq                          NaN           NaN                    NaN   NaN   
mean                     0.127803      0.769767               0.620930   NaN   
std                      0.059222      0.302954               0.327958   NaN   
min                      0.036791      0.100000               0.100000   NaN   
25%                      0.082071      0.500000               0.500000   NaN   
50%                      0.116762      1.000000               0.700000   NaN   
75%                      0.157451      1.000000               1.000000   NaN   
max                      0.308346      1.000000               1.000000   NaN   

        execution_tim

In [14]:
test_run.get_aggregate_feedback()

,embedding_cosine_distance,faithfulness,score_string:accuracy,error,execution_time
count,86.000000,86.000000,86.000000,0,86.000000
unique,NaN,NaN,NaN,0,NaN
top,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN
mean,0.127803,0.769767,0.620930,NaN,19.343951
std,0.059222,0.302954,0.327958,NaN,5.240861
min,0.036791,0.100000,0.100000,NaN,5.946557
25%,0.082071,0.500000,0.500000,NaN,15.802171
50%,0.116762,1.000000,0.700000,NaN,19.639004
75%,0.157451,1.000000,1.000000,NaN,22.089945


# Comparing with other indexing strategies

The index used above retrieves the raw documents based on a single vector per document. It doesn't perform any additional chunking. You can try changing the chunking parameters when generating the index.

## Customizing Chunking

The simplest change you can make to the index is configure how you split the 

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def transform_docs(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
    yield from splitter.split_documents(docs)


# Used for the cache
transformation_name = "recursive-text-cs4k-ol200"

retriever_factory = langchain_docs.retriever_factories["basic"]

chunked_retriever = retriever_factory(
    embeddings,
    transform_docs=transform_docs,
    transformation_name=transformation_name,
    search_kwargs={"k": 4},
)

In [20]:
chunked_results = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, retriever, llm=llm),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-spotless-rhythm-97' at:
http://localhost/o/00000000-0000-0000-0000-000000000000/projects/p/0fe51c4e-b79a-4edd-8443-0fecf8bc220e?eval=true

View all tests for Dataset LangChain Docs Q&A at:
http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b
[------------------------------------------------->] 86/86
 Eval quantiles:
        embedding_cosine_distance  score_string:accuracy  faithfulness error  \
count                   86.000000              86.000000     86.000000     0   
unique                        NaN                    NaN           NaN     0   
top                           NaN                    NaN           NaN   NaN   
freq                          NaN                    NaN           NaN   NaN   
mean                     0.131206               0.574419      0.755814   NaN   
std                      0.057896               0.322558      0.319413   NaN   
min                      0.

In [21]:
chunked_results.get_aggregate_feedback()

,embedding_cosine_distance,score_string:accuracy,faithfulness,error,execution_time
count,86.000000,86.000000,86.000000,0,86.000000
unique,NaN,NaN,NaN,0,NaN
top,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN
mean,0.131206,0.574419,0.755814,NaN,16.795995
std,0.057896,0.322558,0.319413,NaN,6.063518
min,0.035323,0.100000,0.100000,NaN,4.930493
25%,0.089841,0.300000,0.500000,NaN,13.534636
50%,0.119418,0.600000,1.000000,NaN,16.058093
75%,0.158104,0.850000,1.000000,NaN,18.820127


## Parent Document Retriever

This indexing technique chunks documents and generates 1 vector per chunk.
At retrieval time, the K "most similar" chunks are fetched, then the full parent documents are returned for the LLM to reason over.

This ensures the chunk is surfaced in its full natural context. It also can potentially improve the initial retrieval quality since the similarity scores are scoped to individual chunks.

Let's see if this technique is effective in our case.

In [ ]:
retriever_factory = langchain_docs.retriever_factories["parent-doc"]

# Indexes the documents with the specified embeddings
parent_doc_retriever = retriever_factory(embeddings)

In [ ]:
parent_doc_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, parent_doc_retriever, llm=llm),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

In [ ]:
parent_doc_test_run.get_aggregate_feedback()

## HyDE

HyDE (Hypothetical document embeddings) refers to the technique of using an LLM
to generate example queries that my be used to retrieve a doc. By doing so, the resulting embeddings are automatically "more aligned" with the embeddings generated from the query. This comes with an additional indexing cost, since each document requires an additoinal call to an LLM while indexing.

In [ ]:
retriever_factory = langchain_docs.retriever_factories["hyde"]

retriever = retriever_factory(embeddings)

In [ ]:
hyde_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, retriever),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

In [ ]:
hyde_test_run.get_aggregate_feedback()

# Comparing Embeddings

We've been using off-the-shelf GTE-Base embeddings so far to retrieve the docs, but
you may get better results with other embeddings. You could even try fine-tuning embedddings on your own documentation and evaluating here.

Let's compare our results so far to OpenAI's embeddings.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings()

In [ ]:
openai_retriever = langchain_docs.retriever_factories["basic"](openai_embeddings)

In [ ]:
openai_embeddings_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, openai_retriever),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

In [ ]:
openai_embeddings_test_run.get_aggregate_feedback()

## Comparing Models

We used Anthropic's Claude-2 model in our previous tests, but lets try with some other models.

You can swap in any LangChain LLM within the response generator below.
We'll try a long-context llama 2 model first (using Ollama).

In [29]:
from langchain.chat_models import ChatOllama

# A llama2-based model with 128k context
# (in theory) In practice, we will see how well
# it actually leverages that context.
ollama = ChatOllama(model="yarn-llama2:7b-128k")

In [31]:
# We'll go back to the GTE embeddings for now

retriever_factory = langchain_docs.retriever_factories["basic"]
retriever = retriever_factory(embeddings)

In [32]:
ollama_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, llm=ollama, retriever=retriever),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-complicated-motion-26' at:
http://localhost/o/00000000-0000-0000-0000-000000000000/projects/p/10089cd0-a4dc-45e1-8c6d-66143324944c?eval=true

View all tests for Dataset LangChain Docs Q&A at:
http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b
[>                                                 ] 0/86

Chain failed for example 9ee18a2e-34a9-4578-8afe-40d98ade8f7b with inputs {'question': 'how do i initialize OpenAIAnthropicVectorStore?'}
Error Type: ValueError, Message: Ollama call failed with status code 500. Details: llama runner process has terminated


[>                                                 ] 1/86


KeyboardInterrupt

Chain failed for example c737a33a-fb32-4b85-92cd-1b204c698231 with inputs {'question': 'whats the difference between run house and click house'}
Error Type: ValueError, Message: Ollama call failed with status code 500. Details: llama runner process has terminated


[>                                                 ] 2/86

## Changing the prompt in the response generator

The default prompt was tested primariily on OpenAI's gpt-3.5 model. When switching models, you may get better results if you modify the prompt. Let's try a simple one.

In [ ]:
from langchain import hub
from langchain.schema.output_parser import StrOutputParser

In [ ]:
prompt = hub.pull("wfh/rag-simple")

In [ ]:
generator = prompt | ChatAnthropic(model="claude-2", temperature=1) | StrOutputParser()
new_chain = chain_factory(response_synthesizer=generator, retriever=openai_retriever)

In [ ]:
claude_simple_prompt_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(
        chain_factory, response_synthesizer=generator, retriever=retriever
    ),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

## Testing Agents

Agents use an LLM to decide actions and generate responses. There are two obvious ways they could potentially succeed where the approaches above fail:
- The above chains do not "rephrase" the user query. It could be that the rephrased question will result in more relevant documents.
- The above chains must respond based on a single retrieval step. Agents can iteratively query the retriever or subdivide the query into different parts to synthesize at the end. Our dataset has a number of questions that require information from different documents - if the

Let's evaluate to see whether the "plausible" statements above are worth the tradeoffs. We will use the basic retriever as a tool for them.

In [22]:
from typing import List, Tuple

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.tools import tool
from langchain.tools.render import format_tool_to_openai_function

# This is used to tell the model how to best use the retriever.


@tool
def search(query, callbacks=None):
    """Search the LangChain docs with the retriever."""
    return retriever.get_relevant_documents(query, callbacks=callbacks)


tools = [search]

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
assistant_system_message = """You are a helpful assistant tasked with answering technical questions about LangChain. \
Use tools (only if necessary) to best answer the users questions. Do not make up information if you cannot find the answer using your tools."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", assistant_system_message),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "agent_scratchpad": lambda x: format_to_openai_functions(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(..., extra={"widget": {"type": "chat"}})


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False).with_types(
    input_type=AgentInput
)


class ChainInput(BaseModel):
    question: str


def mapper(input: dict):
    return {"input": input["question"], "chat_history": []}


agent_executor = (mapper | agent_executor | (lambda x: x["output"])).with_types(
    input_type=ChainInput
)

In [23]:
oai_functions_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=agent_executor,
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-mealy-lip-56' at:
http://localhost/o/00000000-0000-0000-0000-000000000000/projects/p/fc7df61c-9062-40d4-a823-5051cf3f551c?eval=true

View all tests for Dataset LangChain Docs Q&A at:
http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b
[--------->                                        ] 18/86

Chain failed for example 1f97ba11-f475-4974-b179-3be47f5882ec with inputs {'question': 'How do i run llama 2 in langchain'}
Error Type: InvalidRequestError, Message: This model's maximum context length is 16385 tokens. However, your messages resulted in 19672 tokens (19627 in the messages, 45 in the functions). Please reduce the length of the messages or functions.


[------------------------------------->            ] 65/86

Chain failed for example cb004261-4831-4477-a4f8-3d7b69919ff6 with inputs {'question': 'Show me an example using Weaviate, but customizing the VectorStoreRetriever to return the top 10 k nearest neighbors. '}
Error Type: InvalidRequestError, Message: This model's maximum context length is 16385 tokens. However, your messages resulted in 20974 tokens (20929 in the messages, 45 in the functions). Please reduce the length of the messages or functions.


[------------------------------------------------->] 86/86
 Eval quantiles:
        embedding_cosine_distance  faithfulness  score_string:accuracy  \
count                   84.000000     81.000000              84.000000   
unique                        NaN           NaN                    NaN   
top                           NaN           NaN                    NaN   
freq                          NaN           NaN                    NaN   
mean                     0.122338      0.696296               0.521429   
std                      0.063324      0.312027               0.329685   
min                      0.028011      0.100000               0.100000   
25%                      0.077464      0.500000               0.100000   
50%                      0.109392      0.700000               0.500000   
75%                      0.159934      1.000000               0.700000   
max                      0.293347      1.000000               1.000000   

                                   

## Assistant

OpenAI provides a hosted agent service through their Assistants API. 

You can connect your LangChain retriever to an OpenAI's Assistant API and evaluate its performance. Let's test below:

In [26]:
import json

from langchain.agents import AgentExecutor
from langchain.tools import tool
from langchain_experimental.openai_assistant import OpenAIAssistantRunnable


@tool
def search(query, callbacks=None) -> str:
    """Search the LangChain docs with the retriever."""
    docs = retriever.get_relevant_documents(query, callbacks=callbacks)
    return json.dumps([doc.dict() for doc in docs])


tools = [search]

agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain docs assistant",
    instructions="You are a helpful assistant tasked with answering technical questions about LangChain.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)


assistant_exector = (
    (lambda x: {"content": x["question"]})
    | AgentExecutor(agent=agent, tools=tools)
    | (lambda x: x["output"])
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
assistant_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=assistant_exector,
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-long-night-86' at:
http://localhost/o/00000000-0000-0000-0000-000000000000/projects/p/da7b7f81-926c-4155-95fe-0c9a91b7983b?eval=true

View all tests for Dataset LangChain Docs Q&A at:
http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b
[>                                                 ] 0/86

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[>                                                 ] 1/86

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[----------------------------->                    ] 52/86

Chain failed for example d42a702e-3a92-4cb6-9148-8fa9a5f6c062 with inputs {'question': 'How do I load Youtube transcripts and CSV documents?'}
Error Type: BadRequestError, Message: Error code: 400 - {'error': {'message': "Expected tool outputs for call_ids ['call_xOTr7Y2mMgAvBngL55HOluqO', 'call_lSnkgQ94fb7wEMyP9LupkNEn'], got ['call_lSnkgQ94fb7wEMyP9LupkNEn']", 'type': 'invalid_request_error', 'param': None, 'code': None}}


[------------------------------------------------->] 86/86
 Eval quantiles:
        score_string:accuracy  faithfulness  embedding_cosine_distance  \
count                     0.0           0.0                  85.000000   
unique                    NaN           NaN                        NaN   
top                       NaN           NaN                        NaN   
freq                      NaN           NaN                        NaN   
mean                      NaN           NaN                   0.129928   
std                       NaN           NaN                   0.061485   
min                       NaN           NaN                   0.028841   
25%                       NaN           NaN                   0.083430   
50%                       NaN           NaN                   0.118423   
75%                       NaN           NaN                   0.155580   
max                       NaN           NaN                   0.343805   

                                   

In [28]:
assistant_test_run.get_aggregate_feedback()

,score_string:accuracy,faithfulness,embedding_cosine_distance,error,execution_time
count,0.0,0.0,85.000000,1,86.000000
unique,NaN,NaN,NaN,1,NaN
top,NaN,NaN,NaN,"Error code: 400 - {'error': {'message': ""Expec...",NaN
freq,NaN,NaN,NaN,1,NaN
mean,NaN,NaN,0.129928,NaN,28.175951
std,NaN,NaN,0.061485,NaN,10.704861
min,NaN,NaN,0.028841,NaN,6.088222
25%,NaN,NaN,0.083430,NaN,22.164567
50%,NaN,NaN,0.118423,NaN,26.051888
75%,NaN,NaN,0.155580,NaN,34.683894
